> ## Calibrate Vicon coordinates to 3dMD coordinates

$$\bm{x}_{3dmd} = s \bm{R} \bm{x}_{vicon} + \bm{t}$$

If only want to dry-run, set `is_output` as `False`.

Otherwise, the calibration parameters will be outputted to `mesh4d/config/calibrate/` folder:

| Variables | Contents                                |
| --------- | --------------------------------------- |
| `s.npy`   | Storing the scaling rate $s$.           |
| `r.npy`   | Storing the rotation matrix $\bm R$.    |
| `t.npy`   | Storing the translation vector $\bm t$. |

_P.S. Before calibration, 3dMD mesh, key points extracted from 3dMD mesh, and corresponding Vicon key points data need to be prepared. Noted that the key points from 3dMD mesh is extracted with `kps_extract.py` and saved as `points_3dmd.npy`._

In [18]:
# is_output = True
is_output = False

### Preparation and Data Loading

Variables related to 3dMD system:

| Variables     | Contents                                                    |
| ------------- | ----------------------------------------------------------- |
| `points_3dmd` | Loaded key points from 3dMD system as `numpy` (N, 3) array. |
| `pcd_3dmd`    | Loaded key point converted as `open3d` point cloud.         |
| `pvpcd_3dmd`  | Loaded key point converted as `pyvista` point cloud.        |

Variables related to Vicon system:

| Variables      | Contents                                                     |
| -------------- | ------------------------------------------------------------ |
| `points_vicon` | Loaded key points from Vicon system as `numpy` (N, 3) array. |
| `pcd_vicon`    | Loaded key point converted as `open3d` point cloud.          |
| `pvpcd_vicon`  | Loaded key point converted as `pyvista` point cloud.         |

In [19]:
# add root folder of the project to path
import sys
sys.path.insert(0, '../../..')

In [20]:
# load markers
import numpy as np
points_3dmd = np.load('../../config/calibrate/points_3dmd.npy')

from mesh4d import obj3d
pcd_3dmd = obj3d.np2pcd(points_3dmd)

In [21]:
# load kps
from mesh4d import kps
vicon = kps.MarkerSet()
vicon.load_from_vicon('../../data/6kmh_softbra_8markers_1.csv')
vicon.interp_field()

points_vicon = vicon.get_frame_coord(0)
pcd_vicon = obj3d.np2pcd(points_vicon)

loaded 1 vicon file: ../../data/6kmh_softbra_8markers_1.csv


In [22]:
# plot point clouds before calibration
import pyvista as pv
pv.set_jupyter_backend('pythreejs')

pvpcd_3dmd = obj3d.np2pvpcd(points_3dmd, radius=10, phi_resolution=10, theta_resolution=10)
pvpcd_vicon = obj3d.np2pvpcd(points_vicon, radius=10, phi_resolution=10, theta_resolution=10)

scene = pv.Plotter()
scene.add_mesh(pvpcd_3dmd, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.add_axes(box=True)
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

### Pre-alignment and Calibration

To facilitate accurate calibration, manual pre-alignment (rotation `r_pre`) is implemented to Vicon key points and yields:

| Variables          | Contents                                                |
| ------------------ | ------------------------------------------------------- |
| `points_vicon_pre` | Pre-aligned Vicon key points as `numpy` (N, 3) array    |
| `pcd_vicon_pre`    | Pre-aligned Vicon point cloud as `open3d` point cloud.  |
| `pvpcd_vicon_pre`  | Pre-aligned Vicon point cloud as `pyvista` point cloud. |

And then the Vicon key points is calibrate with 3dMD key points:

| Variables | Contents                          |
| --------- | --------------------------------- |
| `r_cab`   | The estimated rotation matrix.    |
| `s`       | The estimated scaling rate.       |
| `t`       | The estimated translation vector. |

The estimated calibration parameters are applied to the pre-aligned Vicon point cloud and yields:

| Variables          | Contents                                                |
| ------------------ | ------------------------------------------------------- |
| `points_vicon_cab` | Calibrated Vicon key points as `numpy` (N, 3) array    |
| `pvpcd_vicon_cab`  | Calibrated Vicon point cloud as `pyvista` point cloud. |

In [23]:
# pre-alignment
import copy
pcd_vicon_pre = copy.deepcopy(pcd_vicon)
r_pre = pcd_vicon_pre.get_rotation_matrix_from_xyz((-np.pi/2, 0, 0))
pcd_vicon_pre.rotate(r_pre, center=(0, 0, 0))
points_vicon_pre = obj3d.pcd2np(pcd_vicon_pre)

In [24]:
# plot point clouds after pre-alignment
pvpcd_vicon_pre = obj3d.np2pvpcd(points_vicon_pre, radius=10, phi_resolution=10, theta_resolution=10)

scene = pv.Plotter()
scene.add_mesh(pvpcd_vicon_pre, color='Green')
scene.add_mesh(pvpcd_3dmd, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.add_axes(box=True)
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [25]:
# cpd rigid registration
from probreg import cpd
tf_param, _, _ = cpd.registration_cpd(
    pcd_vicon_pre, pcd_3dmd, 'rigid', maxiter=int(1e5), tol=1e-5,# update_scale=False,
)

r_cab = tf_param.rot
s = tf_param.scale
t = tf_param.t

points_vicon_cab = tf_param.transform(points_vicon_pre)

In [26]:
# plot point clouds after calibration
# points_3dmd_cab = obj3d.pcd2np(pcd_3dmd_cab)
pvpcd_vicon_cab = obj3d.np2pvpcd(points_vicon_cab, radius=10, phi_resolution=10, theta_resolution=10)

scene = pv.Plotter()
scene.add_mesh(pvpcd_vicon_cab, color='Gold')
scene.add_mesh(pvpcd_3dmd, color='Blue')
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

### Calibration Validation

In previous steps, pre-alignment rotation `r_pre` and the estimated rotation `r_cab` are all implemented:

$$\bm{x}_{3dmd} = s \bm{R}_{cab} [\bm{R}_{pre} \bm{x}_{vicon}] + \bm{t}
= s [\bm{R}_{cab} \bm{R}_{pre}] \bm{x}_{vicon} + \bm{t}$$

Therefore, their mutual effects are calculated as $\bm{R} = \bm{R}_{cab} \bm{R}_{pre}$ stored as `r`, leading to the complete calibration parameters:

| Variables | Contents                        |
| --------- | ------------------------------- |
| `s`       | The scaling rate $s$.           |
| `r`       | The rotation matrix $\bm R$.    |
| `t`       | The translation vector $\bm t$. |

_P.S. About the combined parameter $\bm M$: [Spatial Transformation Matrices - Rainer Goebel](https://www.brainvoyager.com/bv/doc/UsersGuide/CoordsAndTransforms/SpatialTransformationMatrices.html)_

$$\mathcal T(\bm{x}) = s \bm{R} \bm{x} + \bm{t} = s \bm{M} \bm{x}$$

To verify its effectiveness, these parameters are applied to the original Vicon point cloud and plotted together with the 3dMD mesh & key points:

| Variables         | Contents                                                         |
| ----------------- | ---------------------------------------------------------------- |
| `mesh`            | 3dMD mesh loaded with `pyvista`.                                 |
| `texture`         | 3dMD mesh texture loaded with `pyvista`.                         |
| `pvpcd_vicon_val` | Validation calibrated Vicon point cloud as `open3d` point cloud. |

In [27]:
# combine pre-alignment and registration
# r = np.matmul(r_pre, r_cab)
r = np.matmul(r_cab, r_pre)

In [28]:
# plot 3dmd data before calibrating to vicon
mesh = pv.read('../../data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.obj')
texture = pv.read_texture('../../data/6kmh_softbra_8markers_1/speed_6km_soft_bra.000001.jpg')

scene = pv.Plotter()
scene.add_mesh(mesh, show_edges=True)
scene.add_mesh(pvpcd_3dmd, color='Gold')
scene.add_mesh(pvpcd_vicon, color='Blue')
scene.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [29]:
# plot 3dmd data after calibrating to vicon
from mesh4d import field
s, m = field.transform_rst2sm(r, s, t)

pvpcd_vicon_val = copy.deepcopy(pvpcd_vicon)
pvpcd_vicon_val.transform(m)
pvpcd_vicon_val.scale(s)

scene = pv.Plotter()
scene.add_mesh(mesh, show_edges=True)
scene.add_mesh(pvpcd_3dmd, color='Gold')
scene.add_mesh(pvpcd_vicon_val, color='Blue')
scene.show()

/Users/kirov/Anaconda/anaconda3/lib/python3.8/site-packages/pyvista/core/pointset.py:261: PyvistaDeprecationWarning: You did not specify a value for `inplace` and the default value will be changing to `False` in future versions for point-based meshes (e.g., `PolyData`). Please make sure you are not assuming this to be an inplace operation.
  warnings.warn(DEFAULT_INPLACE_WARNING, PyvistaDeprecationWarning)


Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

### Parameter Saving

If `is_output` is set as `True`, the calibration parameters will be outputted to `mesh4d/config/calibrate/` folder:

| Variables | Contents                                |
| --------- | --------------------------------------- |
| `s.npy`   | Storing the scaling rate $s$.           |
| `r.npy`   | Storing the rotation matrix $\bm R$.    |
| `t.npy`   | Storing the translation vector $\bm t$. |

In [30]:
# save parameters
if is_output:
    np.save('r', r)
    np.save('s', s)
    np.save('t', t)